In [17]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import pickle
import os
from sklearn.metrics import classification_report, accuracy_score, f1_score, plot_confusion_matrix,ConfusionMatrixDisplay, confusion_matrix
import pickle
os.environ["WANDB_DISABLED"] = "true"

In [18]:
#Loading the Label Encoder 
le = pickle.load(open("Data/label_encoder.pkl", "rb"))

In [19]:
max_length=128
fine_tuned_model_name = 'Models/Final_model/BERT_13classes_final'

In [20]:
#Loading the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(fine_tuned_model_name, do_lower_case=True)

In [21]:
target_names = le.transform(le.classes_)

In [22]:
target_names

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [23]:
def preprocess_function(examples):
    #print(examples['TEXT'])
    return tokenizer(examples["text"], padding=True, truncation=True)
    #return toknizer(examples["text"],padding='max_length',truncation=True, max_length=max_length)

In [24]:
load_model = BertForSequenceClassification.from_pretrained(fine_tuned_model_name, num_labels=13)
load_model.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [61]:
def get_new_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
   # print(inputs)
    # perform inference to our model
    outputs = load_model(**inputs)
    #print(outputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1).argmax()
    #print(probs.item())
    #print(outputs[0].softmax(1).argmax())
    # executing argmax function to get the candidate label
    return probs.item()
    #return probs.argmax().item()

In [64]:
labels_dict = {'y': 'yes-answers',
               'y-d': 'yes-answers-explanations',
               'n': 'no-answers',
               'n-d': 'no-answers-explanations',
               'sno': 'statements-non-opinion',
               'so': 'statements-opinion',
               'ack': 'acknowledgments',
               'dno': "other answers",
               'query': 'interviewee-initiated questions',
               'ft': 'thanking',
               'fa': 'apologies', 
               'fe': 'explanations',
               'fp': 'conventional'
              }

### Get predictions

In [77]:
text = "i think this works well"


In [78]:
print("The label is: ", labels_dict[le.inverse_transform([get_new_prediction(text)])[0]])

The label is:  statements-opinion
